# Open AI functions parameter

In [26]:
from openai import OpenAI

from util import local_settings
from IPython.display import display, HTML

display(HTML(f"""
<h1>LCEL Introduction</h1>
✅ Libraries loaded successfully <br>
✅ OpenAI Key assigned (<font color="LightGreen">...{local_settings.OPENAI_API_KEY[10:-15]}...</font>)
"""))

llm_client = OpenAI(api_key=local_settings.OPENAI_API_KEY)



## Function creation

In [21]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## Function definition for OpenAI (schema)

In [22]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

## Message

In [23]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [29]:
response = llm_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        functions=functions
    )

response

ChatCompletion(id='chatcmpl-8O6rwIQ14btc6Mj69gs8BYOioo4VX', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700756968, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))

In [33]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [34]:
response.choices[0].message.content

In [35]:
response.choices[0].message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [36]:
response.choices[0].message.function_call.arguments

'{\n  "location": "Boston, MA"\n}'

In [37]:
arguments = json.loads(response.choices[0].message.function_call.arguments)
print(arguments)

{'location': 'Boston, MA'}


## You need call the function

In [38]:
get_current_weather(arguments)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'